In [1]:
import sys
import os
from pathlib import Path
import pandas as pd
from collections import Counter
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
#pip install lxml

In [85]:
folder_path = "../../tmp/out4/"
xml_files = [filename for filename in os.listdir(folder_path) if filename.endswith(".xml")][:2]
xml_files

['Liu et al_2020.grobid.tei.xml', 'Sakane et al_2020.grobid.tei.xml']

In [104]:
file_path = os.path.join(folder_path, xml_files[0])
with open(file_path, 'r') as tei:
    soup = BeautifulSoup(tei, 'lxml')

/Users/lindahemmann/miniconda3/envs/pytorch-env-cpu/lib/python3.11/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [103]:
soup.title
#soup.abstract.find_all('p')

In [88]:
def display_hierarchy(tag, level=0):
    # Print the tag name and value with proper indentation
    print("  " * level + f"Tag: {tag.name}") #, Value: {tag.get_text()}")

    # Recursively process child tags
    for child in tag.find_all(recursive=False):
        display_hierarchy(child, level + 1)

In [89]:
#display_hierarchy(soup)

In [90]:
def is_reference(tag):
    # You may need to adjust this function based on the actual structure of your references
    return (tag.name == 'biblstruct') or (tag.name == 'listbibl')

In [91]:
# Find and remove all reference (ref) tags
def remove_references(tag):
    # You may need to adjust this function based on the actual structure of your references
    return tag.name in ['biblstruct', 'listbibl', 'ref']
for ref_tag in soup.find_all(remove_references):
    ref_tag.extract()

In [92]:
filename_without_extension = os.path.basename(file_path).rsplit(".")[0]
#os.path.splitext(os.path.basename(file_path))#os.path.splitext(os.path.basename(file_path))[0]
filename_without_extension

'Liu et al_2020'

In [105]:
soup.title

<title level="a" type="main">The age-dependent effect of high-dose X-ray radiation on NFκB signaling, structure, and mechanical behavior of the intervertebral disc</title>

In [106]:
# Find all div tags
div_tags = soup.find_all('div')
output_folder = '../../tmp/processed'

# Extract and print the content within each div
# Create a new XML document
new_xml = BeautifulSoup(features="xml")

# Create a root element for the new XML document
root = new_xml.new_tag("root")
new_xml.append(root)
root.append(soup.title)

for abstract_p in [p.get_text(separator=' ', strip=True) for p in soup.abstract.find_all('p')]:
    a_tag = new_xml.new_tag("abstract")
    a_tag.string = abstract_p
    root.append(a_tag)

for div_tag in div_tags:
    #main_text = [child for child in div_tag.children if not is_reference(child)]
    paragraphs = [p.get_text(separator=' ', strip=True) for p in div_tag.find_all('p')]

    # Print the list of paragraphs
    #print(f"Content within div (excluding references):")
    #for paragraph in paragraphs:
    #    print(paragraph)
    #    print('-' * 30)
    #print('=' * 30)

    # Add each paragraph as a new 'p' element under the root
    for paragraph in paragraphs:
        p_tag = new_xml.new_tag("p")
        p_tag.string = paragraph
        root.append(p_tag)

# Save the new XML document to a file
# Get the original filename (without extension)
filename_without_extension = os.path.basename(file_path).rsplit(".")[0]

# Construct the output XML file path using the original filename
print(str(new_xml))
output_path = os.path.join(output_folder, f'{filename_without_extension}.xml')
with open(output_path, 'w', encoding='utf-8') as output_file:
    output_file.write(str(new_xml))

        
    # Concatenate the main text elements
    #main_text_content = '\n'.join(str(item) for item in main_text).strip()
    #print(f"Content within div (excluding references):\n{main_text_content}\n{'='*30}\n")
    #div_content = div_tag.get_text(separator='\n', strip=True)
    #print(f"Content within div:\n{div_content}\n{'='*30}\n")

<?xml version="1.0" encoding="utf-8"?>
<root><title level="a" type="main">The age-dependent effect of high-dose X-ray radiation on NFκB signaling, structure, and mechanical behavior of the intervertebral disc</title><abstract>Purpose: Ionizing radiation damages tissue and provokes inflammatory responses in multiple organ systems. We investigated the effects of high-dose X-ray radiation on the molecular inflammation and mechanical function of the intervertebral disc (IVD). Methods: Functional spine units (FSUs) containing the vertebrae-IVDs-vertebrae structure extracted from 1-month, 6-month, and 16-month-old NFκB-luciferase reporter mice and from 6-month-old myeloid differentiation factor 88 (MyD88)-null mice. After a preconditioning period in culture, the FSUs were subjected a single dose of ionizing X-ray radiation at 20 Gys, and then NFκB expression was monitored. The IVDs were then subjected to mechanical testing using dynamic compression, glycosaminoglycan (GAG) quantification, an

In [7]:
for tag in soup.find_all(True):
    print(f"Tag: {tag.name}") #, Value: {tag.get_text()}")

Tag: html
Tag: body
Tag: tei
Tag: teiheader
Tag: filedesc
Tag: titlestmt
Tag: title
Tag: funder
Tag: orgname
Tag: funder
Tag: orgname
Tag: orgname
Tag: publicationstmt
Tag: publisher
Tag: availability
Tag: licence
Tag: sourcedesc
Tag: biblstruct
Tag: analytic
Tag: author
Tag: persname
Tag: forename
Tag: forename
Tag: surname
Tag: affiliation
Tag: orgname
Tag: orgname
Tag: address
Tag: settlement
Tag: region
Tag: country
Tag: affiliation
Tag: orgname
Tag: orgname
Tag: address
Tag: settlement
Tag: region
Tag: country
Tag: author
Tag: persname
Tag: forename
Tag: surname
Tag: affiliation
Tag: orgname
Tag: orgname
Tag: orgname
Tag: address
Tag: settlement
Tag: region
Tag: country
Tag: author
Tag: persname
Tag: forename
Tag: forename
Tag: surname
Tag: email
Tag: idno
Tag: affiliation
Tag: orgname
Tag: orgname
Tag: address
Tag: settlement
Tag: region
Tag: country
Tag: affiliation
Tag: orgname
Tag: orgname
Tag: address
Tag: settlement
Tag: region
Tag: country
Tag: affiliation
Tag: orgname
Tag:

In [15]:
file_path = os.path.join(folder_path, xml_files[0])
# Parse the TEI XML file
tree = ET.parse(file_path)
root = tree.getroot()

In [20]:
soup.text
#root.attrib

"\n\n\n\nThe age-dependent effect of high-dose X-ray radiation on NFκB signaling, structure, and mechanical behavior of the intervertebral disc\n\nWashington University Musculoskeletal Research Center\n\n\nNational Institutes of Health\nNIH\n\n\n\n\n\n\n\n\n\n\nJenniferWLiu\n\nDepartment of Biomedical Engineering\nWashington University in St. Louis\n\nSt. Louis\nMO\nUSA\n\n\n\nDepartment of Orthopaedic Surgery\nWashington University in St. Louis\n\nSt. Louis\nMO\nUSA\n\n\n\n\nSytsePiersma\n\nDivision of Rheumatology\nDepartment of Medicine\nWashington University in St. Louis\n\nSt. Louis\nMO\nUSA\n\n\n\n\nSimonYTang\nsimon.tang@wustl.edu\n0000-0002-5570-3921\n\nDepartment of Biomedical Engineering\nWashington University in St. Louis\n\nSt. Louis\nMO\nUSA\n\n\n\nDepartment of Orthopaedic Surgery\nWashington University in St. Louis\n\nSt. Louis\nMO\nUSA\n\n\n\nDepartment of Materials Science and Mechanical Engineering\nWashington University in St. Louis\n\nSt. Louis\nMO\nUSA\n\n\n\n\n\nO

In [18]:
for child in root:

    print(child.tag, child.attrib)

{http://www.tei-c.org/ns/1.0}teiHeader {'{http://www.w3.org/XML/1998/namespace}lang': 'en'}
{http://www.tei-c.org/ns/1.0}text {'{http://www.w3.org/XML/1998/namespace}lang': 'en'}


In [26]:
root[1][1].text

'\n\n\t\t\t'

In [8]:
tei_data = []

for filename in xml_files:
    file_path = os.path.join(folder_path, filename)
    # Parse the TEI XML file
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Extract and process relevant information from the XML
    # Modify this part according to the structure of your TEI XML files
    # For example, if your data is within <body> tags, you can access it as root.find('.//body').text
    tei_content = root.text  # Replace this line with your specific extraction logic

    # Append the extracted data to the list
    tei_data.append(tei_content)

In [14]:
root.keys()


['{http://www.w3.org/XML/1998/namespace}space',
 '{http://www.w3.org/2001/XMLSchema-instance}schemaLocation']

In [ ]:
def read_tei_xml_files(folder_path):
    tei_data = []

    # Iterate through files in the specified folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".xml"):
            file_path = os.path.join(folder_path, filename)

            # Parse the TEI XML file
            tree = ET.parse(file_path)
            root = tree.getroot()

            # Extract and process relevant information from the XML
            # Modify this part according to the structure of your TEI XML files
            # For example, if your data is within <body> tags, you can access it as root.find('.//body').text
            tei_content = root.text  # Replace this line with your specific extraction logic

            # Append the extracted data to the list
            tei_data.append(tei_content)

    return tei_data

# Example usage
folder_path = '/path/to/your/xml/files'
tei_data_list = read_tei_xml_files(folder_path)